In [27]:
from metric import ETRMetrics
import similarity_search
import faiss
import json
import os
import re
from prompts import ETRRules, PromptTemplate
from tqdm import tqdm

In [28]:
from transformers import pipeline

# pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [29]:
def load_data(path) :
    length = 0
    embedding_path ="data/data_embedded/"+path
    original_path ="data/original_data/"+path+"/sources"
    index_path = ".index"
    data_path = ".json"
    
    index = similarity_search.load_embedded_dataset(embedding_path+index_path)
    json_obj = similarity_search.load_order_data(embedding_path+data_path)
    
    for filename in os.listdir(original_path):
        if 'test' in filename:
            with open(os.path.join(original_path, filename), 'r', encoding='utf-8') as file:
                for line in file :
                    length += 1
    
    return index,json_obj,length

In [30]:
etr_politic_index, etr_politic_data, etr_politic_length = load_data("etr-fr-politic")
etr_index, etr_data, etr_length = load_data("etr-fr")
etr_index, etr_data, etr_length = load_data("wikilarge-fr")

In [31]:
nbr_shot = 5

In [32]:
a = '''
Le facile à lire et à comprendre (FALC) est une méthode qui a pour but de traduire un langage classique en langage compréhensible par tous. 
Le texte ainsi simplifié peut être compris par les personnes handicapées mentales, 
mais aussi par d’autres comme les personnes dyslexiques, malvoyantes, les personnes âgées, les personnes qui maîtrisent mal le français.

Voici des principes de clarté et d’accessibilité suivants pour que tu puisse rendre un texte facile à lire et à comprendre  :
- Donne uniquement les informations essentielles. Évite la surcharge d’informations.
- Présente les informations dans un ordre logique et facile à suivre.
- Mets en avant l’information principale dès le début.
- Regroupe ensemble les informations qui parlent du même sujet.
- Répète les informations importantes si cela aide à la compréhension.
- Utilise des phrases courtes et simples.
- Choisis des mots faciles à comprendre.
- Explique clairement les mots difficiles, et répète l’explication si besoin.
- Utilise un langage adapté aux personnes concernées.
- Emploie le même mot pour parler de la même chose tout au long du texte.
- Évite les idées abstraites, les métaphores et les comparaisons complexes.
- Ne recours pas à des mots étrangers ou peu connus sans explication.
- Ne pas utiliser de mots contractés ou de style "texto".
- Adresse-toi directement au lecteur, de manière claire et accessible.
- Veille à ce que les pronoms soient toujours clairs et non ambigus.
- Privilégie des formulations positives plutôt que négatives.
- Utilise la voix active autant que possible.
- Choisis une ponctuation simple.
- Utilise des puces ou des numéros pour les listes, plutôt que des virgules.
- Écris les nombres en chiffres (ex. : 1, 2, 3), pas en lettres.
- Explique les sigles dès leur première apparition.\n- N’utilise pas d’abréviations non expliquées.
- Écris les dates en toutes lettres pour plus de clarté.
- Limite l’usage des pourcentages ou grands nombres, et explique-les simplement.
- N’utilise pas de caractères spéciaux inutiles.
- Utilise des exemples concrets pour illustrer les idées complexes.
- Privilégie des exemples issus de la vie quotidienne.'''

c='''\n\n Réécris le texte suivant en prenant en compte touts les principes de clarté et d’accessibilité précédents. Garde le contexte tel quel, n'ajoute ni titre ni section supplémentaire ni saut de ligne. \n\n Entoure ta réponse par des balises "@@@" comme dans les exemples précédents.'''

In [22]:
shot_prompt = "\n\n Voici quelques exemples de transformation de texte en texte facile à lire et à comprendre.\n\n"
pred = []
ref = []
src = []

for i in tqdm(range(etr_length)) :
    idx = similarity_search.get_most_similar_id(etr_index, i, nbr_shot+1)
    prompt = shot_prompt
    
    for j,k in enumerate(idx) :
        prompt += "Exemple "+str(j)+"\n"
        prompt += "Texte original :\n"
        prompt += '"'+etr_data[k]['original']+'"\n'
        prompt += "Traduction en texte facile à lire et à comprendre :\n"
        prompt += "@@@"+etr_data[k]['falc']+"@@@"
        prompt += '\n\n'
        
    messages = [
        {"role": "system", "content": a+prompt+c+"Traduction en texte facile à lire et à comprendre :\n"},
        {"role": "user", "content": etr_data[i]['original']},
    ]
    
    response = pipe(messages,max_new_tokens=8000)
    match = re.search('@@@(.*?)@@@', response[0]['generated_text'][-1]['content'],re.DOTALL)
    pred.append(match.group(1) if match else response[0]['generated_text'][-1]['content'].replace("@@@", ""))
    ref.append(etr_data[i]['falc'])
    src.append(etr_data[i]['original'])
    # print("#################################################################")
    # print("System")
    # print(a+prompt+c)
    # print("\nInput")
    # print(etr_politic_data[i]['original'])
    # print(response[0]['generated_text'][-1]['content'])
        


  2%|▏         | 1/53 [00:10<09:22, 10.81s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


@@@La cloche sonne. C'est la fin de l'année. Les élèves sont contents. Ils sortent du collège en courant. Lucas traverse la cour en souriant. « Salut, bonnes vacances! » crient ses amis. Lucas pense : « Bonnes vacances, c'est facile à dire, mais qu'est-ce que je vais faire pendant l'été? » Le magasin de ses parents reste ouvert, donc ils ne partent pas en vacances. Ses amis partent dans des endroits différents. Et Manon, la petite-fille des voisins, n'est pas là non plus. Elle a appelé Lucas de l'aéroport pour lui dire qu'elle partait pour l'Afrique avec ses parents. Elle allait faire un safari-photo! Lucas est heureux pour elle, mais il est un peu triste de ne pas la revoir. Il doit l'admettre : Manon lui manque beaucoup.@@@


  4%|▍         | 2/53 [00:18<07:48,  9.18s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


@@@Lucas arrive chez lui. Il jette son cartable dans l'entrée. Il mange trois pains au lait et une tablette de chocolat. Puis il s'allonge sur le canapé avec sa console de jeux. Il est fatigué. Son téléphone portable sonne. Lucas pense que c'est sa maman. Mais c'est Manon. Elle dit : « Au secours, Lucas! J'ai besoin de toi! ». Lucas est surpris. Il demande : « Manon, où es-tu? ». Mais Manon ne répond pas. Il y a un silence. Lucas est inquiet. Comment peut-il aider Manon, qui se trouve très loin de chez lui?@@@


  6%|▌         | 3/53 [00:28<07:45,  9.31s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

@@@Le père de Jules dit : « La bouteille est finie, il faut en chercher une nouvelle dans le garage ». Sa mère se lève et part dans le garage. « Papy, on peut faire une grande balade en vélo tous ensemble aujourd'hui! », demande Jules. Il est ravi mais il pense à Bou qui est dans le jardin. « Et Bou? », demande Jules. Mais avant qu'il ne puisse répondre, il entend un cri strident. C'est sa mère! Elle est dans le garage! Jules et son père se précipitent pour voir ce qu'il se passe. Sa mère est perchée sur un escabeau et elle continue à crier. Il y a une souris dans le garage! Son père prend un balai et essaie de la chasser mais ça ne marche pas.@@@


 98%|█████████▊| 52/53 [06:46<00:10, 10.51s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


@@@Et puis, d'un coup, tout se déroule très vite. Bou attrape la souris en trois bonds. La mère de Jules retrouve son calme et son papa lâche son balai. « Bravo Bou! » s'exclame Jules. « Bravo Bou? » répètent ses parents. « Oui, je l'adore! C'est mon ami! » « Tu l'aimes vraiment ce chat? » demande sa mère. « Oui, je te l'ai déjà dit! Il t'a débarrassée de cette souris! » « Il n'a pas tort, fait remarquer son papa. » « Vous êtes donc tous les deux contre moi? » s'exclame la mère de Jules. « Et il n'a pas de famille, ce Bou? » « Non, il a été abandonné par son ancienne famille et maintenant il est tout seul. » Jules supplie sa mère : « Maman, est-ce que Bou peut rester? » Sa mère regarde son mari qui hausse les épaules puis répond : « D'accord, Bou peut rester mais c'est parce qu'il nous a débarrassés de cette souris! Et je te préviens, c'est toi qui passeras l'aspirateur pour enlever ses poils qui traînent et à la première griffure sur mes rideaux il prend la porte. On est bien d'accord

100%|██████████| 53/53 [06:52<00:00,  7.79s/it]

@@@C'est un jour super! Jules est heureux. Son papa a préparé les vélos et sa maman a préparé un pique-nique. Ils sont tous partis. Bou est dans le panier à vélo de maman. Il est drôle avec ses babines au vent. C'est la meilleure promenade que Jules a faite. Ils ont passé une très bonne journée ensemble, papa, maman, Bou et Jules!@@@


# Resultats

# Llama3.1 8B : etr-fr-politic
System :
Le facile à lire et à comprendre (FALC) est une méthode qui a pour but de traduire un langage classique en langage compréhensible par tous. Le texte ainsi simplifié peut être compris par les personnes handicapées mentales, mais aussi par d’autres comme les personnes dyslexiques, malvoyantes, les personnes âgées, les personnes qui maîtrisent mal le français.

Voici des principes de clarté et d’accessibilité suivants pour que tu puisse rendre un texte facile à lire et à comprendre  :
- Donne uniquement les informations essentielles. Évite la surcharge d’informations.
- Présente les informations dans un ordre logique et facile à suivre.
- Mets en avant l’information principale dès le début.
- Regroupe ensemble les informations qui parlent du même sujet.
- Répète les informations importantes si cela aide à la compréhension.
- Utilise des phrases courtes et simples.
- Choisis des mots faciles à comprendre.
- Explique clairement les mots difficiles, et répète l’explication si besoin.
- Utilise un langage adapté aux personnes concernées.
- Emploie le même mot pour parler de la même chose tout au long du texte.
- Évite les idées abstraites, les métaphores et les comparaisons complexes.
- Ne recours pas à des mots étrangers ou peu connus sans explication.
- Ne pas utiliser de mots contractés ou de style "texto".
- Adresse-toi directement au lecteur, de manière claire et accessible.
- Veille à ce que les pronoms soient toujours clairs et non ambigus.
- Privilégie des formulations positives plutôt que négatives.
- Utilise la voix active autant que possible.
- Choisis une ponctuation simple.
- Utilise des puces ou des numéros pour les listes, plutôt que des virgules.
- Écris les nombres en chiffres (ex. : 1, 2, 3), pas en lettres.
- Explique les sigles dès leur première apparition.
- N’utilise pas d’abréviations non expliquées.
- Écris les dates en toutes lettres pour plus de clarté.
- Limite l’usage des pourcentages ou grands nombres, et explique-les simplement.
- N’utilise pas de caractères spéciaux inutiles.
- Utilise des exemples concrets pour illustrer les idées complexes.
- Privilégie des exemples issus de la vie quotidienne.

 Voici quelques exemples de transformation de texte en texte facile à lire et à comprendre.

Exemple 0
Texte original :
"En me faisant confance, vous voterez pour les travailleurs de ce pays, qui font notre prospérité collective. Les salariés du privé gagneront plus grâce au triplement de la prime annuelle défscalisée. Il sera possible de placer ses RTT et jours de congés sur un compte afn de se les faire payer ou de les utiliser comme congés au cours de sa vie. Les impôts continueront de baisser, avec la suppression de la redevance télé. Il sera possible de transmettre le fruit d’une vie de travail sans impôt, à ses enfants comme à d’autres membres de la famille avec une baisse des impôts sur les successions. Pour ramener le plus grand nombre sur le chemin du travail et atteindre le plein emploi, un horizon enfn accessible pour notre pays, les droits et devoirs seront renforcés : le versement automatique et simplifé des aides sociales sera mis en place, en contrepartie de la justifcation d’une activité d’insertion pour les percevoir. "
Traduction en texte facile à lire et à comprendre :
@@@En votant pour moi, vous votez pour les travailleurs. La prime annuelle sans impôt sera augmentée. Les jours de congés pourront être payés ou utilisés plus tard. Les impôts continueront de baisser. La taxe pour ceux qui ont une télé sera supprimée. Nous baisserons les impôts sur les héritages. Les droits et les obligations pour le chômage seront renforcés. Le versement des aides sociales sera simplifié. Il faudra par contre justifier d’une formation ou d’une recherche d’emploi pour recevoir les aides sociales. @@@

Exemple 1
Texte original :
"TRAVAILLEURS FRANÇAIS ET IMMIGRÉS: UN MÊME CAMP ! Sans travailleurs immigrés, aucun hôpital, aucun chantier du bâtiment ou de nettoyage, aucune usine ne fonctionnerait. Ces travailleurs sont bien plus utiles que les capitalistes Arnault ou Bolloré, ou leurs serviteurs du monde politique. Ils doivent avoir le droit de vote. Zemmour et Le Pen voudraient opposer les Français aux étrangers, et s’en prennent en particulier aux musulmans. Ils voudraient faire de la France une forteresse qui, tout en accueillant les milliardaires, rejette les pauvres. Ne nous laissons pas diviser. Les seuls bénéficiaires en seraient les capitalistes."
Traduction en texte facile à lire et à comprendre :
@@@Les travailleurs français et ceux des autres pays sont tous frères. Les travailleurs immigrés sont les travailleurs originaires d’un autre pays. Les travailleurs immigrés sont bien plus utiles que les patrons. Ils doivent avoir le droit de vote. Zemmour et Le Pen veulent opposer les travailleurs français et immigrés. Mais si les travailleurs sont divisés, les patrons en profitent. @@@

Exemple 2
Texte original :
"En me faisant confance, vous voterez pour une République et une Nation plus fortes dans notre Europe. Nous continuerons d'investir dans notre défense et de sceller le pacte Armées-Nation. Nous maîtriserons mieux nos frontières et conditionnerons l'obtention des titres de séjour long à la maîtrise de notre langue et la connaissance de notre culture. Nous poursuivrons notre action pour bâtir une Europe-puissance, capable, par son indépendance, de se défendre et de peser sur le cours du monde. "
Traduction en texte facile à lire et à comprendre :
@@@En votant pour moi, vous votez pour une France forte dans l’Europe. Nous continuerons à donner des moyens à nos armées. Nous contrôlerons mieux nos frontières. Il faudra savoir parler français pour avoir un titre de séjour. Nous continuerons notre travail pour construire une Europe forte. @@@

Exemple 3
Texte original :
"En me faisant confance, vous voterez pour vivre mieux où que vous habitiez, à la campagne comme à la ville, dans l'hexagone comme dans l'Outre-mer. Vivre en sécurité avec le renforcement de nos armées et la mise en œuvre du doublement de la présence des policiers et gendarmes sur la voie publique ainsi que le recrutement des magistrats et personnels nécessaires à une justice rapide et efcace. Vivre en bonne santé avec le recul des déserts médicaux, la mise en place de bilan de santé gratuits aux âges clés et la poursuite de l’investissement dans notre hôpital public. La vie sera aussi plus simple avec un guichet unique et un numéro unique pour toutes les démarches administratives. "
Traduction en texte facile à lire et à comprendre :
@@@En votant pour moi, vous votez pour vivre mieux chez vous Vous vivrez en sécurité : ● il y aura 2 fois plus de policiers et de gendarmes ● des juges seront embauchés. Vous vivrez en bonne santé : ● il y aura moins de régions sans médecin ● à certains âges les contrôles de santé seront gratuits ● nous continuerons à améliorer l’hôpital public. Les démarches administratives seront plus simples. @@@

Exemple 4
Texte original :
"En me faisant confance, au fond, c’est pour vous que vous voterez. J’ai appris de ces cinq ans à la tête du pays qu’il n’y avait d’ambition que collective, de réussite que coopérative. C’est pourquoi vous serez associés à chaque grande décision. Les temps de crise que nous traversons, les grandes transformations que nous vivons, bouleversent tout et inquiètent tout le monde. Je veillerai à ne laisser personne au bord du chemin, à défendre sans relâche nos valeurs de liberté, d'égalité, de fraternité, de laïcité, à vous protéger à chaque instant. "
Traduction en texte facile à lire et à comprendre :
@@@En votant pour moi, c’est pour vous que vous votez. Vous participerez aux grandes décisions. Je ferai tout pour ne laisser personne de côté. Vive la République ! Vive la France !  @@@



 Réécris le texte suivant en prenant en compte touts les principes de clarté et d’accessibilité précédents. Garde le contexte tel quel, n'ajoute ni titre ni section supplémentaire ni saut de ligne. Entoure ta réponse par des balises "@@@" comment dans les exemples précédents.

Input :
En me faisant confance, vous voterez pour préparer notre avenir. Parce que nous continuerons d’investir pour être le premier grand pays à sortir du pétrole, du gaz, du charbon. Parce que nous mettrons en place une taxe carbone écologique aux frontières de l’Europe, nos enfants et nos petits-enfants pourront vivre demain dans une planète respirable et un environnement préservé. Parce que la priorité sera donnée à l’école, les savoirs fondamentaux davantage enseignés, nos professeurs plus libres, plus respectés et mieux rémunérés, notre lycée professionnel devenu une voie d’excellence, chaque Français, quel que soit son origine, son milieu social, sera formé pour afronter le monde et choisir sa vie.

Génération :
En votant pour moi, vous préparez notre avenir. Nous continuerons à investir pour sortir du pétrole, du gaz et du charbon. Nous mettrons une taxe carbone pour protéger l'environnement. Nos enfants et nos petits-enfants pourront vivre dans un monde propre. Nous donnerons la priorité à l'école. Les élèves apprendront plus de choses importantes. Les professeurs seront mieux payés et respectés. Les lycées professionnels seront une excellente option pour tous. Chaque Français, quelle que soit sa famille ou son milieu social, sera formé pour réussir dans la vie.

In [35]:
# Llama3.1 8B : etr-fr-politic

metric = ETRMetrics("fr")
res = metric(pred, ref, src)
res

{'rouge1': 42.0078,
 'rouge2': 18.8091,
 'rougeL': 30.2703,
 'rougeLsum': 38.5028,
 'sari': 45.88,
 'bleu': 11.1873,
 'bertscore_f1_rescaled': 34.2322,
 'bertscore_recall_rescaled': 38.8621,
 'bertscore_precision_rescaled': 29.7825,
 'bertscore_f1': 75.3535,
 'bertscore_recall': 77.1183,
 'bertscore_precision': 73.7198,
 'kmre': 0.0,
 'lix': 39.2555,
 'compression_ratio': -9.7652,
 'novelty': 50.5754,
 'srb': 44.0513,
 'n_samples': 33}

## Mistral 7B v0.3 : etr-fr-politic
En me faisant confance, vous voterez pour préparer notre avenir. Parce que nous continuerons d’investir pour être le premier grand pays à sortir du pétrole, du gaz, du charbon. Parce que nous mettrons en place une taxe carbone écologique aux frontières de l’Europe, nos enfants et nos petits-enfants pourront vivre demain dans une planète respirable et un environnement préservé. Parce que la priorité sera donnée à l’école, les savoirs fondamentaux davantage enseignés, nos professeurs plus libres, plus respectés et mieux rémunérés, notre lycée professionnel devenu une voie d’excellence, chaque Français, quel que soit son origine, son milieu social, sera formé pour afronter le monde et choisir sa vie.  

En votant pour moi, vous votez pour notre avenir. Nous continuerons à investir pour être le premier grand pays à arrêter d'utiliser le pétrole, le gaz, le charbon. Nous mettrons en place une taxe écologique sur les frontières de l'Europe, pour que nos enfants et nos petits-enfants puissent vivre demain dans une planète respirable et un environnement préservé. Nous donnerons plus d'importance à l'école, enseignant les connaissances fondamentales, en faisant plus de respect à nos professeurs et en les payant mieux, en faisant de notre lycée professionnel une voie d'excellence. Chaque Français, quelle que soit son origine ou son milieu social, sera formé pour affronter le monde et choisir sa vie. 

In [45]:
# Mistral 7B v0.3 : etr-fr-politic

metric = ETRMetrics("fr")
res = metric(pred, ref, src)
res

{'rouge1': 40.4767,
 'rouge2': 17.6837,
 'rougeL': 28.7977,
 'rougeLsum': 36.8768,
 'sari': 43.2192,
 'bleu': 10.6939,
 'bertscore_f1_rescaled': 29.443,
 'bertscore_recall_rescaled': 33.7311,
 'bertscore_precision_rescaled': 25.3717,
 'bertscore_f1': 73.5588,
 'bertscore_recall': 75.198,
 'bertscore_precision': 72.069,
 'kmre': 0.0,
 'lix': 42.1738,
 'compression_ratio': -17.2877,
 'novelty': 33.6001,
 'srb': 41.983,
 'n_samples': 33}

In [16]:
# Mistral 7B v0.3 : etr-fr

metric = ETRMetrics("fr")
res = metric(pred, ref, src)
res

{'rouge1': 31.2069,
 'rouge2': 11.0857,
 'rougeL': 22.0696,
 'rougeLsum': 30.2973,
 'sari': 39.9568,
 'bleu': 6.2861,
 'bertscore_f1_rescaled': 26.9123,
 'bertscore_recall_rescaled': 33.7707,
 'bertscore_precision_rescaled': 20.6662,
 'bertscore_f1': 72.6104,
 'bertscore_recall': 75.2128,
 'bertscore_precision': 70.3078,
 'kmre': 0.0,
 'lix': 27.086,
 'compression_ratio': 23.7809,
 'novelty': 26.4571,
 'srb': 35.6674,
 'n_samples': 53}

In [23]:
# Llama3.1 8B : etr-fr

metric = ETRMetrics("fr")
res = metric(pred, ref, src)
res

{'rouge1': 32.0899,
 'rouge2': 12.603,
 'rougeL': 23.0497,
 'rougeLsum': 31.3397,
 'sari': 41.9421,
 'bleu': 7.4727,
 'bertscore_f1_rescaled': 28.9533,
 'bertscore_recall_rescaled': 36.2835,
 'bertscore_precision_rescaled': 22.1617,
 'bertscore_f1': 73.3753,
 'bertscore_recall': 76.1532,
 'bertscore_precision': 70.8675,
 'kmre': 0.0,
 'lix': 25.4033,
 'compression_ratio': 22.8153,
 'novelty': 33.5831,
 'srb': 37.1032,
 'n_samples': 53}

# Test feedback

In [33]:
f='''Voici maintenant des exemples négatif de texte transformé en facile à lire et à comprendre. Ce sont des exemples qui ont été transformé en facile à lire et à comprendre de la mauvaise manière. Je veux que tu y fasse vraiment attention pour ne pas reproduire les mêmes erreurs que ces exemples négatifs. Voici les exemples :

Exemple négatif 0
Texte original :
Toujours avec son caddie, Léa se rend directement au rayon des œufs. Cela lui fait tout drôle d'être seule dans le magasin, même si ce n'est que pour quelques minutes. À cet instant, elle a vraiment l'impression d'être grande et elle est très fière. Comme elle fait souvent les courses avec sa mère, elle sait exactement où se trouvent les œufs et sait quoi prendre. Elle passe d'abord devant le lait avant de les trouver. Elle saisit une boîte et vérifie qu'aucun d'entre eux n'est cassé, comme sa mère le lui a montré. Tout est bon, elle pose la boîte avec précaution dans le caddie.

Texte mal transformé en facile à lire et à comprendre :
Léa va chercher les œufs. Elle est seule dans le magasin. Elle est fière. Elle sait où sont les œufs. Elle sait quoi prendre. Elle passe devant le lait. Elle prend une boîte d'œufs. Elle vérifie si les œufs sont cassés. Tout est bon. Elle met la boîte dans le caddie.

Explication des erreurs :
- Il aurait fallut mettre en avant l’information principale dès le début, à savoir que "Léa est heureuse" car elle est allée faire les courses comme une grande, et pas qu'elle est allée chercher des oeufs.
- Les pronoms ne sont pas assez clair. Dans le facile à lire et à comprendre négatif, les phrases commençent par "Elle" au lieu de "Léa". "Léa" aurait dû être utilisé pour que le locuteur ne se perde pas dans l'histoire.
- Il y a des informations qui sont inutiles, et qui n'apporte aucun changement à la compréhension de l'histoire : "Elle sait quoi prendre. Elle passe devant le lait.". Il aurait fallut supprimer cette partie. Il faut donc mieux compresser l'information.
- Les informations sont répétées: "Elle prend une boîte d'œufs.","Elle met la boîte dans le caddie". Une seule des deux phrases est suffisante pour la compréhension du texte.

Texte correctement transformé en facile à lire et à comprendre :
Léa est heureuse. « Super ! Je vais chercher les œufs, toute seule ! » Léa trouve les œufs très vite. Les œufs sont fragiles. Léa fait attention de ne pas casser les œufs.
\n'''

In [35]:
shot_prompt = "\n\n Voici quelques exemples de transformation de texte en texte facile à lire et à comprendre.\n\n"
pred = []
ref = []
src = []

for i in tqdm(range(etr_length)) :
    idx = similarity_search.get_most_similar_id(etr_index, i, nbr_shot+1)
    prompt = shot_prompt
    
    for j,k in enumerate(idx) :
        prompt += "Exemple "+str(j)+"\n"
        prompt += "Texte original :\n"
        prompt += '"'+etr_data[k]['original']+'"\n'
        prompt += "Traduction en texte facile à lire et à comprendre :\n"
        prompt += "@@@"+etr_data[k]['falc']+"@@@"
        prompt += '\n\n'
        
    messages = [
        {"role": "system", "content": a+prompt+f+c+"Traduction en texte facile à lire et à comprendre :\n"},
        {"role": "user", "content": etr_data[i]['original']},
    ]
    
    response = pipe(messages,max_new_tokens=8000)
    print(response[0]['generated_text'][-1]['content'])
    match = re.search('@@@(.*?)@@@', response[0]['generated_text'][-1]['content'],re.DOTALL)
    pred.append(match.group(1) if match else response[0]['generated_text'][-1]['content'].replace("@@@", ""))
    ref.append(etr_data[i]['falc'])
    src.append(etr_data[i]['original'])
    print("#################################################################")
    print("System")
    print(a+prompt+f+c)
    print("\nInput")
    print(etr_data[i]['original'])
    print("\nOriginal FALC\n")
    print(etr_data[i]['falc'])
    
        

  2%|▏         | 1/53 [00:11<10:17, 11.88s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


@@@La cloche sonne. C'est la fin de l'année. Les élèves sont contents. Ils sortent du collège en criant de joie. Lucas traverse la cour en souriant. Ses amis lui disent bon voyage et lui font des signes de la main. Lucas pense : « C'est facile de dire bonnes vacances, mais qu'est-ce que je vais faire pendant l'été? » Sa famille ne part pas car le magasin de ses parents est ouvert tout l'année. Ses amis partent en vacances. Manon, la petite-fille des voisins, n'est pas là non plus. Elle a appelé Lucas de l'aéroport pour lui dire qu'elle partait pour l'Afrique avec ses parents. Elle fait un safari-photo! Lucas est heureux pour elle, mais il est un peu triste de ne pas la voir. Il doit l'avouer : Manon lui manque beaucoup.@@@
#################################################################
System
Le facile à lire et à comprendre (FALC) est une méthode qui a pour but de traduire un langage classique en langage compréhensible par tous. Le texte ainsi simplifié peut être compris par les per

  4%|▍         | 2/53 [00:23<09:49, 11.56s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


@@@Lucas rentre chez lui. Il jette son cartable et se met à manger des pains au lait. Il mange trois et une tablette de chocolat. Ensuite, il s'allonge sur le canapé avec sa console de jeux. Il est fatigué! Son téléphone sonne. C'est peut-être sa maman. Mais Lucas pense : « Elle pourrait attendre ce soir. » Pourtant, il décide de répondre. Il pose la console et prend le téléphone. Le nom de Manon apparaît à l'écran. Lucas est surpris. Il répond vite. Manon hurle : « Lucas, j'ai besoin de ton aide! ». Lucas est médusé. Il demande : « Manon, où es-tu? ». Il n'y a pas de réponse. Il y a un silence. Lucas reste bouche bée. Il ne sait pas comment aider Manon. Elle est loin, très loin.
#################################################################
System
Le facile à lire et à comprendre (FALC) est une méthode qui a pour but de traduire un langage classique en langage compréhensible par tous. Le texte ainsi simplifié peut être compris par les personnes handicapées mentales, mais aussi par 

  6%|▌         | 3/53 [00:34<09:32, 11.46s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

@@@Jules et son père sont heureux de passer la journée ensemble. « On va faire une balade en vélo! » Mais Jules a oublié son ami Bou. Il veut retourner le chercher. Mais avant, il entend un cri strident. C'est sa mère! Elle est dans le garage et elle hurle. Jules et son père se précipitent pour voir ce qui se passe. Là, ils voient une souris! Sa mère est perchée sur un escabeau et elle continue à crier. Son père essaie de chasser la souris avec un balai, mais il n'y arrive pas.@@@
#################################################################
System
Le facile à lire et à comprendre (FALC) est une méthode qui a pour but de traduire un langage classique en langage compréhensible par tous. Le texte ainsi simplifié peut être compris par les personnes handicapées mentales, mais aussi par d’autres comme les personnes dyslexiques, malvoyantes, les personnes âgées, les personnes qui maîtrisent mal le français.

Voici des principes de clarté et d’accessibilité suivants pour que tu puisse ren

 98%|█████████▊| 52/53 [07:11<00:11, 11.11s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


@@@Bou surgit et attrape la souris en trois bonds. Maman retrouve son calme et papa lâche son balai. « Bou! » s'exclame Jules admiratif. « Tu l'aimes vraiment ce chat, soupire sa mère. » « Oui, je te l'ai dit, c'est mon ami! » « Regarde, il t'a débarrassée de cette souris! » « Avec Bou à la maison, tu n'en auras plus une qui traîne! » Papa fait remarquer : « Il n'a pas tort. » « Vous êtes donc tous les deux contre moi? » s'exclame la mère de Jules. « Et il n'a pas de famille, ce Bou? » « Non, il a été abandonné par son ancienne famille et maintenant il est tout seul. » Jules supplie : « Maman, est-ce que Bou peut rester? » Sa mère regarde son mari qui hausse les épaules. « D'accord, Bou peut rester. » « C'est bien parce qu'il nous a débarrassés de cette souris! » « Et je te préviens, c'est toi qui passeras l'aspirateur pour enlever ses poils qui traînent. » « Et à la première griffure sur mes rideaux, il prend la porte. » « On est bien d'accord! » s'exclame Jules en serrant sa mère dan

100%|██████████| 53/53 [07:18<00:00,  8.28s/it]

@@@C'est un super jour! Jules est heureux. Son papa a préparé les vélos et sa maman a préparé un pique-nique. Jules, son papa et sa maman sont partis. Bou est dans le panier à vélo de maman! Il est drôle avec ses babines au vent. C'est la meilleure journée que Jules a passée avec sa famille. Ils sont tous ensemble : papa, maman, Bou et Jules.@@@
#################################################################
System
Le facile à lire et à comprendre (FALC) est une méthode qui a pour but de traduire un langage classique en langage compréhensible par tous. Le texte ainsi simplifié peut être compris par les personnes handicapées mentales, mais aussi par d’autres comme les personnes dyslexiques, malvoyantes, les personnes âgées, les personnes qui maîtrisent mal le français.

Voici des principes de clarté et d’accessibilité suivants pour que tu puisse rendre un texte facile à lire et à comprendre  :
- Donne uniquement les informations essentielles. Évite la surcharge d’informations.
- Prése

In [36]:
# Llama3.1 8B : etr-fr + negative examples
metric = ETRMetrics("fr")
res = metric(pred, ref, src)
res

{'rouge1': 33.0605,
 'rouge2': 12.7021,
 'rougeL': 23.7062,
 'rougeLsum': 32.5867,
 'sari': 41.6422,
 'bleu': 7.4916,
 'bertscore_f1_rescaled': 28.6623,
 'bertscore_recall_rescaled': 34.4401,
 'bertscore_precision_rescaled': 23.2071,
 'bertscore_f1': 73.2662,
 'bertscore_recall': 75.4633,
 'bertscore_precision': 71.2588,
 'kmre': 0.0,
 'lix': 24.9991,
 'compression_ratio': 30.8044,
 'novelty': 31.1834,
 'srb': 37.5723,
 'n_samples': 53}